In [22]:
import cv2
import pandas as pd
import numpy as np

In [23]:
image = cv2.imread('./temp.jpg', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (250,  250))

In [24]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
 
# Returns true if two rectangles(l1, r1)
# and (l2, r2) overlap
def do_overlap(l1, r1, l2, r2):
     
    # if rectangle has area 0, no overlap
    if l1.x == r1.x or l1.y == r1.y or r2.x == l2.x or l2.y == r2.y:
        return False
     
    # If one rectangle is on left side of other
    if l1.x > r2.x or l2.x > r1.x:

        return False
 
    # If one rectangle is above other
    if r1.y < l2.y or r2.y < l1.y:
        return False

    return True
def mean(l1_array):
    x_new = 0
    y_new = 0
    for i in range(len(l1_array)):
        x_new += l1_array[i].x
        y_new += l1_array[i].y
    x_new /= len(l1_array)
    y_new /= len(l1_array)
    return Point(int(x_new), int(y_new))



In [25]:
def generate_window_points(width, height, window_size):
    width_1 = width - window_size + 1
    height_1 = height - window_size + 1
    points = []
    if(width_1 < 0 or height_1 < 0):
        return None
    for i in range(width_1):
        for j in range(height_1):
            points += [[i, j]]
    points = np.array(np.array(points))
    return points
def generate_window(img,x, y, window_size):
    '''
    x is the row index
    y is the column index
    '''
    # updated the code. 
    img_new = img[x:x+window_size, y:y+window_size]
    return img_new


def windows(img, min_wind, ratio):
    '''
    it is assumed that the image is of a grayscale one.
    '''
    sizes = []
    temp = min_wind
    while(temp < max(img.shape)):
        sizes += [temp]
        temp *= ratio
    return sizes

def gen_and_resize(sizes, windows_list, img, resize_shape):
    '''
    resize_shape -> tuple
    '''
    windows = []
    rectangles = []
    total_windows = 0
    for i in windows_list:
        total_windows += len(i)
    done = 0
    for i in range(len(sizes)):
        size_now = sizes[i]
        windows_list_now = windows_list[i]
        rectangles += [[Point(j[0], j[1]), Point(j[0] + size_now - 1, j[1] + size_now -1)] for j in windows_list_now]
        windows += [cv2.resize(generate_window(img, j[0], j[1], size_now), resize_shape) for j in windows_list_now]
        done += len(windows_list_now)
        print(f'{(done/total_windows)*100} % done')
    return rectangles, windows
def return_all_windows(image, window_size, resize_shape, ratio):
    sizes = windows(image, window_size, ratio)
    width = image.shape[0]
    height = image.shape[1]
    windows_list = [generate_window_points(width, height, i) for i in sizes]
    rectangles, windows_new = gen_and_resize(sizes, windows_list, image, resize_shape)
    return rectangles, windows_new


In [26]:
window_size = 19
resize_shape = (19, 19)
ratio = 2

In [27]:
rectangles, windows_new = return_all_windows(image, window_size, resize_shape, ratio)

38.550627063651795 % done
71.04548807827015 % done
92.9801817804167 % done
100.0 % done


In [28]:
len(rectangles)

139619

In [29]:
len(windows_new)

139619

In [30]:
def do_overlap_api(rectangles_1,rectangles_2):
    return do_overlap(rectangles_1[0], rectangles_1[1], rectangles_2[0], rectangles_2[1])
rand_idx = np.random.randint(0, len(rectangles)-1, size = 50)
rand_idx = np.unique(rand_idx)

In [31]:
rand_idx.shape

(50,)

In [32]:
positives = [rectangles[i] for i in rand_idx]

bins = []
count = 0
for i in range(len(positives)):
    rect_1 = positives[i]
    for j in range(i, len(positives)):
        rect_2 = positives[j]  
        found_bin_1 = 0
        found_bin_2 = 0
        match_12 = do_overlap_api(rect_1, rect_2)
        for bin in range(len(bins)): 
            for rect_3 in bins[bin]:
                if(not found_bin_1):
                    if(do_overlap_api(rect_1, rect_3)):
                        bins[bin] += [rect_1]
                        found_bin_1 = 1
                if(not found_bin_2):
                    if(do_overlap_api(rect_2, rect_3)):
                        bins[bin] += [rect_2]
                        found_bin_2 = 1
                    pass
                if(found_bin_1 and found_bin_2):
                    break
                # try breaking the complete loop
        if(not found_bin_1 and not found_bin_2):
            if(match_12):
                bins += [[rect_1, rect_2]]
            else:
                bins += [[rect_1]]
                bins += [[rect_2]]
        elif(not found_bin_1 and found_bin_2):
            bins += [[rect_1]]
        elif(found_bin_1 and not found_bin_2):
            bins += [[rect_2]]

        count += 1


In [33]:
bins 

[[[<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point at 0x123ce17e0>, <__main__.Point at 0x123ce1840>],
  [<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point at 0x123efb3a0>, <__main__.Point at 0x123efb400>],
  [<__main__.Point a

In [34]:
count = 0
for bin in bins[0]:
    count += 1
print(count)

2363


In [35]:
count

2363

In [36]:
len(bins)

19

In [37]:
bin_0 = bins[0]
bin_0_l = [i[0] for i in bin_0]
bin_0_r = [i[1] for i in bin_0]

In [38]:
mean_l = mean(bin_0_l)
mean_r = mean(bin_0_r)

In [39]:
img_new = cv2.rectangle(image, (mean_l.x, mean_l.y), (mean_r.x, mean_r.y),color = (255, 0, 0), thickness=2)

In [40]:
cv2.imwrite('./img_new.jpg', img_new)

True